In [ ]:
import nltk
from py2neo import *
import pandas as pd
from pprint import pprint
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import spacy
from spacy import displacy
import en_core_web_sm
from nltk import pos_tag
from itertools import *

In [ ]:
def connect():
    global graph
    graph = Graph("bolt://localhost:7687", auth = ("neo4j", "soham"))
    tx = graph.begin()
    print('Connected...')
connect()

In [ ]:
def ask_question():
    global question
    question = input("INPUT: ")
    print("\n")
    tokenize()

In [ ]:
def tokenize():
    global filtered_question
    question_tokenized = word_tokenize(question)
    stop_words = set(stopwords.words('english'))
    filtered_question = [w for w in question_tokenized if not w in stop_words]
    filtered_question = []
    for w in question_tokenized:
        if w not in stop_words:
            filtered_question.append(w)
    tag()

In [ ]:
def tag():
    global ner, tags
    nlp = en_core_web_sm.load()
    doc = nlp(question)
    ner = [(X.text, X.label_) for X in doc.ents]
    print(question)
    print(filtered_question)
    for token in doc:
        print((token.text, token.pos_, token.tag_, token.dep_))
    displacy.render(doc)
    tags = pos_tag(filtered_question)
    print('All tags: ',tags)
    print('Length of the list: ',len(tags))
    groups = groupby(tags, key=lambda x: x[1])
    names_tagged = [[w for w,_ in words] for tag,words in groups if tag=="NNP"]
    print('Tagged names are: ',names_tagged)
    names = [" ".join(name) for name in names_tagged if len(name)>=2]
    parms_builder()

In [ ]:
def parms_builder():
    global parms, parms_2
    if len(ner) == 1:
        if (ner[0][1] == 'GPE') or (ner[0][1] == 'LOC'):
            if (ner[0][0] == "US") or (ner[0][0] == "USA"):
                country_ = 'United States'
            elif (ner[0][0] == "UK"):
                country_ = 'United Kingdom'
            else:
                country = ner[0][0]
                parms = {}
                parms["country"] = country
                print(parms)
        elif (ner[0][1] == 'ORG'):
            org = ner[0][0]
            parms = {}
            parms["org"] = org
            print(parms)
        elif (ner[0][1] == 'PERSON'):
            person = ner[0][0]
            parms = {}
            parms["person"] = person
            print(parms)
    elif len(ner) > 1:
        name1 = ner[0][0]
        name2 = ner[1][0]
        parms_2 = {"name1":name1, "name2":name2}
        print(parms_2)
    query_picker()

In [ ]:
def query_picker():
    
    label_p = "(p:NER_Person)"
    label_o = "(o:NER_Organization)"
    label_l = "(l:NER_Location)"
    works = "-[r:WORKS_AT]->"
    lives = "-[:LIVES_IN]->"

    query1 = '''
    match {} {} {}
    where p.value IN $person
    return p.value as Name,r.value as Works_as,o.value as at
    '''.format(label_p,works,label_o)

    query1_1 = '''
    match {} {} {}
    where o.value IN $org
    return o.value as Organization, collect(distinct p.value) as Person, r.value as Position
    '''.format(label_p,works,label_o)

    query2 = '''
    match (p:NER_Person{value:$person})-[r:LIVES_IN]-(l:NER_Location)
    return p.value as Person, l.value as STAYS_AT
    '''

    query3 = '''
    MATCH (p1:NER_Person:Tag{ value: $name1 }),(p2:NER_Person:Tag{ value: $name2 }), p = shortestPath((p1)-[*..15]-(p2))
    RETURN p1.value as Person1, p2.value as Person2, p as Relation
    '''

    query4 = '''
    MATCH (p:NER_Person)-[w:LIVES_IN]->(o:NER_Location)
    RETURN p.value as Person ,o.value as Lives_in
    '''.format(label_p,lives,label_o)

    query5 = '''
    MATCH (p:NER_Person)-[:LIVES_IN]->(l:NER_Location), (p)-[w:WORKS_AT]-(o:NER_Organization)
    RETURN p.value as Person, l.value as Lives_in, o.value as Works_at, w.AS as Position
    '''

    query6 = '''
    MATCH (s:Sentence)-[st:SENTENCE_TAG_OCCURRENCE]->(n:TagOccurrence), (s)-[h:HAS_TAG]-(p:NER_Person), (s)-[h]-(o:NER_Organization)
    where n.value IN ["said","says","think","thinks"] AND (p.value in $names OR o.value in $org)
    return s.text as Sentence, p.value as Person
    '''
    for word,tag in tags:
        if word in ['work','do']:
            verb = word
            query = query1
            print('We have a verb: ',verb)
            print(query)
            print(graph.run(query,parms).data())
        elif word in ['works','at']:
            verb = word
            print('We have an org: ',word)
            print(graph.run(query1_1,parms).to_table())
        elif word in ['live','reside','stay']:
            verb = word
            print('We have a verb: ',verb)
            print(graph.run(query2,parms).data())
        elif word in ['related','relation']:
            verb = word
            print('We have a verb: ',verb)
            print(graph.run(query3,parms_2).data())
        elif word in ['lives']:
            verb = word
            print('We have a verb: ',verb)
            print(graph.run(query4).to_table())
        elif word in ['everyone']:
            verb = word
            print('We have a verb (s): ',verb)
            print(graph.run(query5).to_table()) 
        elif word in ['think', 'says', 'say']:
            verb = word
            print('We have a verb: ',verb)
            print(graph.run(query6,parms).to_table()) 

In [ ]:
ask_question()